In [1]:
import json
import os
import pandas as pd
from shoeboxer.schedules import default_schedules
from shoeboxer.batch import batch_sim
from pathlib import Path
import numpy as np

from archetypal import settings

# Check if we are running on Windows or Linux using os
if os.name == "nt":
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("C:/EnergyPlusV22-2-0")
else:
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("/usr/local/EnergyPlus-22-2-0")

with open(f"app/template-defaults.json", "r") as f:
    template_defaults = json.load(f)

n_samples = 50
df = pd.DataFrame([template_defaults for _ in range(n_samples)])
df["wwr"] = 0.4
df["width"] = 3
df["perim_depth"] = 5
df["core_depth"] = 4
df["orientation"] = 0
df["roof_2_footprint"] = 1
df["ground_2_footprint"] = 1
df["orientation"] = np.linspace(0, np.pi, n_samples)
for i in range(12):
    df[f"shading_{i}"] = 0
scheds = default_schedules[0]
results = batch_sim(
    features=df,
    timeseries=scheds,
    climate="data/epws/city_epws_indexed/cityidx_0001_USA_NY-New York Central Prk Obs Belv.725033_TMY3.epw",
)


c:\tools\Anaconda3\envs\ml-for-bem-final\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
INFO:shoeboxer.geometry_utils:New perimeter depth: 5.0
INFO:shoeboxer.geometry_utils:New core depth: 4.0
INFO:shoeboxer.geometry_utils:Shifting core origin to perimeter edge; moving 0.0 m in y axis
INFO:shoeboxer.geometry_utils:Updating adiabatic surfaces.
INFO:shoeboxer.geometry_utils:Maximum shading point is 0.0
INFO:shoeboxer.geometry_utils:Placing shading around center point [1.5 0. ]
INFO:ShoeBox:Building idf for data\sim_results\6fbaf6ff-de8c-4825-a7d6-f914d8b3e404\6fbaf6ff-de8c-4825-a7d6-f914d8b3e404.epjson
INFO:ShoeBox:EnergyPlus Starting
INFO:ShoeBox:EnergyPlus, Version 22.2.0-c249759bad, YMD=2023.11.16 14:19
INFO:ShoeBox:Converted input file format. Exiting.
INFO:ShoeBox:EnergyPlus Run Time=00hr 00min

KeyValue                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Core  \
Name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Heating   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    2018-01-01   
id                                   n_floors height building_width building_length wwr_S wwr_E wwr_N wwr_W cop_cooling CoolingSetpoint PeopleDensity VentilationPerPerson RoofMass SlabRValue EconomizerSettings cop_heating LightingPowerDensity Infiltration VentilationMode FacadeRValue WindowUValue RecoverySettings HeatingSetpoint EquipmentPowerDensity VentilationPerArea FacadeMass RoofRValue WindowShgc wwr width perim_depth core_depth orientation roof_2_footprint ground_2_footprint shading_0 shading_1 shading_2 shading_3 shading_4 shading_5 shading_6 shading_7 shading_8 shading_9 shading_10 shading_11 shading_12 shading_13 shading_14 shading_15 shading_16 shading_17 shading_18 shading_19 shading_20 shading_21 shading_22 shading_23              
6fbaf6ff-de8c-4825-a7d6-f914d8b3e404 1.0      4.0    10.0           10.0            0.4   0.4   0.4   0.4   2.5         24.0            0.025         0.003                2.0      3.0        0.0                0.9         10.0                 0.0005       1.0             4.0          4.0          0.0              19.0            8.0                   0.0003             2.0        4.0        0.74       0.4 3.0   5.0         4.0        0.0         1.0              1.0                0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0         15.320362   
6d0ef213-6f75-4341-b870-0a15aabe6051 1.0      4.0    10.0           10.0            0.4   0.4   0.4   0.4   2.5         24.0            0.025         0.003                2.0      3.0        0.0                0.9         10.0    